# OpenAI and LangChain Integration Example

This notebook demonstrates how to use LangChain with OpenAI's API. We'll show:
1. Setting up the OpenAI client
2. Making a simple completion request
3. Using LangChain's chat models
4. Creating a simple chain

**Note:** You'll need to set your OpenAI API key as an environment variable `OPENAI_API_KEY` to run this notebook.


In [3]:
import os
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()  # This will load variables from a .env file into the environment

# Get your OpenAI API key from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment. Please set it in your .env file.")


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the ChatOpenAI model
model = ChatOpenAI(
    model="gpt-3.5-turbo",  # You can change this to "gpt-4" if you have access
    temperature=0.7
)


In [5]:
# Create a simple prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides concise responses."),
    ("user", "{input}")
])

# Create a simple chain
chain = prompt | model | StrOutputParser()

# Test the chain
response = chain.invoke({"input": "What are the three main components of LangChain?"})
print(response)


The three main components of LangChain are Language Engine, Data Manager, and Application Interface.


In [6]:
# Example of streaming responses
async def do_stream():
    async for chunk in chain.astream({"input": "Explain what an LLM is in exactly 3 sentences."}):
        print(chunk, end="", flush=True)
    print()  # New line at the end

# Run the streaming example
import asyncio
await do_stream()


An LLM, or Master of Laws, is a postgraduate degree typically pursued by individuals who have already completed their JD or equivalent law degree. It allows students to specialize in a particular area of law, such as tax law or international law. The program usually involves coursework, research, and the completion of a thesis or dissertation.


In [7]:
# Create a specialized prompt for CV ranking
cv_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional CV evaluator. Rank CVs on a scale of 1-3:
    1 = Weak (Significant improvements needed)
    2 = Neutral (Meets basic requirements)
    3 = Strong (Exceptional qualifications)
    
    Provide your ranking followed by a brief explanation."""),
    ("user", """Please evaluate this CV:
    {cv_content}""")
])

# Create a CV evaluation chain
cv_chain = cv_prompt | model | StrOutputParser()

# Example CV to test
sample_cv = """
Education:
- Master's in Computer Science from Stanford University (GPA: 3.9)
- Bachelor's in Software Engineering from MIT

Experience:
- Senior Software Engineer at Google (3 years)
  - Led team of 5 engineers on Cloud Platform projects
  - Improved system performance by 40%
  
- Software Developer at Microsoft (2 years)
  - Developed features for Office 365
  - Mentored 3 junior developers

Skills:
- Python, Java, C++, JavaScript
- Machine Learning, Cloud Computing
- Team Leadership, Project Management
"""

# Get the evaluation
response = cv_chain.invoke({"cv_content": sample_cv})
print("CV Evaluation:\n")
print(response)


CV Evaluation:

Ranking: 3

Explanation:
This CV is strong and stands out due to the candidate's impressive education, experience at top tech companies like Google and Microsoft, significant accomplishments, and a diverse set of technical skills. The candidate's leadership experience, project management skills, and proficiency in various programming languages make them a highly qualified candidate. Overall, this CV demonstrates a strong background in computer science and software engineering, making it a top contender for relevant positions.
